In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
# #         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# !rm -rf outputs/*
langs = ['am', 'dz', 'ha', 'ig', 'ma', 'pcm', 'pt', 'sw', 'yo', 'twi', 'kr', 'ts', 'multilingual']
# LANG = 'multilingual'

from sklearn.metrics import f1_score, precision_score, recall_score

PATH = '/kaggle/working/full_outputs/'
if not os.path.exists(PATH):
    os.makedirs(PATH)

def saver(model, split, scores, pred, orig_df):
    
    save_dir = f"{PATH}/{model}"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    df = orig_df.copy()
    
    df['pred'] = pred
    scores_df = pd.DataFrame(scores, columns=['negative', 'neutral', 'positive'])
    df['negative'] = scores_df['negative']
    df['neutral'] = scores_df['neutral']
    df['positive'] = scores_df['positive']
    
    df.to_csv(f"{save_dir}/{LANG}_{split}_output.tsv", sep='\t', index = False)
    
    if split != 'test':
        f1 = f1_score(df['label'],df['pred'], average="weighted")
        recall = recall_score(df['label'], df['pred'], average="weighted")
        precision = precision_score(df['label'], df['pred'], average="weighted")

        m_f1 = f1_score(df['label'],df['pred'], average="macro")
        m_recall = recall_score(df['label'], df['pred'], average="macro")
        m_precision = precision_score(df['label'], df['pred'], average="macro")
        
        with open(f"{save_dir}/{LANG}_{split}_scores.txt", 'w') as output_file:
            output_file.write("WEIGHTED F1\n\navg_precision:{}\n".format(precision)) 
            output_file.write("avg_recall:{}\n".format(recall))
            output_file.write("avg_f1_score:{}\n".format(f1))

            output_file.write("\nMACRO F1\n\nmacro_precision:{}\n".format(m_precision))
            output_file.write("macro_recall:{}\n".format(m_recall))
            output_file.write("macro_f1_score:{}\n".format(m_f1))
        
for LANG in langs:
    print(f'\n{LANG}')
    train_df = pd.read_csv(f'/kaggle/input/semevalafrisenti/comb_data/train/{LANG}_train.tsv', sep = '\t')
    dev_df = pd.read_csv(f'/kaggle/input/semevalafrisenti/comb_data/dev/{LANG}_dev.tsv', sep = '\t')
    test_df = pd.read_csv(f'/kaggle/input/semevalafrisenti/comb_data/test/{LANG}_test.tsv', sep = '\t')

    x_train, x_test, y_train, y_test = train_df['tweet'], dev_df['tweet'], train_df['label'], dev_df['label']
    print(len(x_train),len(y_train),len(x_test),len(y_test))

    x_unk = test_df['tweet']


    #     return df

    print('svm')

    from sklearn.feature_extraction.text import TfidfVectorizer

    vectorizer = TfidfVectorizer()

    tf_x_train = vectorizer.fit_transform(x_train)
    tf_x_test = vectorizer.transform(x_test)
    tf_x_unk = vectorizer.transform(x_unk)

    from sklearn.svm import LinearSVC
    from sklearn.calibration import CalibratedClassifierCV

    svc = LinearSVC(random_state = 0)
    clf = CalibratedClassifierCV(svc) 
    # clf = LinearSVC(random_state = 0)

    clf.fit(tf_x_train, y_train)

    train_pred = clf.predict(tf_x_train)
    train_scores = clf.predict_proba(tf_x_train)
    saver(model = 'svm', split = 'train', scores = train_scores, pred = train_pred, orig_df = train_df)

    dev_pred = clf.predict(tf_x_test)
    dev_scores = clf.predict_proba(tf_x_test)
    saver(model = 'svm', split = 'dev', scores = dev_scores, pred = dev_pred, orig_df = dev_df)

    test_pred = clf.predict(tf_x_unk)
    test_scores = clf.predict_proba(tf_x_unk)
    saver(model = 'svm', split = 'test', scores = test_scores, pred = test_pred, orig_df = test_df)

    # train_pred, train_scores

#     !ls outputs/svm
    print('LR')

    from sklearn.linear_model import LogisticRegression

    clf = LogisticRegression(max_iter = 1000)

    clf.fit(tf_x_train, y_train)

    train_pred = clf.predict(tf_x_train)
    train_scores = clf.predict_proba(tf_x_train)
    saver(model = 'logistic_regression', split = 'train', scores = train_scores, pred = train_pred, orig_df = train_df)

    dev_pred = clf.predict(tf_x_test)
    dev_scores = clf.predict_proba(tf_x_test)
    saver(model = 'logistic_regression', split = 'dev', scores = dev_scores, pred = dev_pred, orig_df = dev_df)

    test_pred = clf.predict(tf_x_unk)
    test_scores = clf.predict_proba(tf_x_unk)
    saver(model = 'logistic_regression', split = 'test', scores = test_scores, pred = test_pred, orig_df = test_df)


#     !ls outputs/logistic_regression
    print('RF')

    from sklearn.ensemble import RandomForestClassifier

    clf = RandomForestClassifier()

    clf.fit(tf_x_train, y_train)

    train_pred = clf.predict(tf_x_train)
    train_scores = clf.predict_proba(tf_x_train)
    saver(model = 'random_forest', split = 'train', scores = train_scores, pred = train_pred, orig_df = train_df)

    dev_pred = clf.predict(tf_x_test)
    dev_scores = clf.predict_proba(tf_x_test)
    saver(model = 'random_forest', split = 'dev', scores = dev_scores, pred = dev_pred, orig_df = dev_df)

    test_pred = clf.predict(tf_x_unk)
    test_scores = clf.predict_proba(tf_x_unk)
    saver(model = 'random_forest', split = 'test', scores = test_scores, pred = test_pred, orig_df = test_df)



am
5984 5984 1497 1497
svm
LR
RF

dz
1651 1651 414 414
svm
LR
RF

ha
14172 14172 2677 2677
svm
LR
RF

ig
10192 10192 1841 1841
svm
LR
RF

ma
5583 5583 1215 1215
svm
LR
RF

pcm
5121 5121 1281 1281
svm


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Pr

LR


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Pr

RF

pt
3063 3063 767 767
svm
LR
RF

sw
1810 1810 453 453
svm
LR


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


RF

yo
8522 8522 2090 2090
svm
LR
RF

twi
3481 3481 388 388
svm
LR
RF

kr
3302 3302 827 827
svm
LR
RF

ts
804 804 203 203
svm
LR
RF

multilingual
63685 63685 13653 13653
svm
LR
RF


In [3]:
# from sklearn.neural_network import MLPClassifier

# clf = MLPClassifier(hidden_layer_sizes=(100), max_iter=100, verbose=True, activation = 'relu',solver='adam',random_state=1)



# clf.fit(tf_x_train, y_train)

# train_pred = clf.predict(tf_x_train)
# train_scores = clf.predict_proba(tf_x_train)
# saver(model = 'mlp', split = 'train', scores = train_scores, pred = train_pred, orig_df = train_df)

# dev_pred = clf.predict(tf_x_test)
# dev_scores = clf.predict_proba(tf_x_test)
# saver(model = 'mlp', split = 'dev', scores = dev_scores, pred = dev_pred, orig_df = dev_df)

# test_pred = clf.predict(tf_x_unk)
# test_scores = clf.predict_proba(tf_x_unk)
# saver(model = 'mlp', split = 'test', scores = test_scores, pred = test_pred, orig_df = test_df)

In [4]:
# from IPython.display import FileLink

# !tar -czvf combined_output_with_test.tar.gz outputs
# FileLink(r'combined_output_with_test.tar.gz')

In [5]:
# !cat outputs/random_forest/*dev*.txt

In [6]:
# import emoji

In [7]:
# output_dir = "/kaggle/working/outputs/formatted_data/er"

# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# new_train_df = train_df.copy()
# new_train_df['tweet'] = train_df['tweet'].apply(lambda x: emoji.demojize(x, delimiters=("", "")))
# new_train_df.to_csv(f"{output_dir}/ha_train_er.tsv", sep = '\t', index = False)

# new_dev_df = dev_df.copy()
# new_dev_df['tweet'] = dev_df['tweet'].apply(lambda x: emoji.demojize(x, delimiters=("", "")))
# new_dev_df.to_csv(f"{output_dir}/ha_dev_er.tsv", sep = '\t', index = False)

# new_test_df = test_df.copy()
# new_test_df['tweet'] = test_df['tweet'].apply(lambda x: emoji.demojize(x, delimiters=("", "")))
# new_test_df.to_csv(f"{output_dir}/ha_test_er.tsv", sep = '\t', index = False)



In [8]:
# print(new_train_df['tweet'][0])

In [9]:
# import re
# def cleaner(tweet):
#     whitespace = re.compile(r"\s+")
#     web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")
#     user = re.compile(r"(?i)@[a-z0-9_]+")
    
# #     tweet = text.replace("@user", "")
#     tweet = whitespace.sub(' ', tweet)
#     tweet = web_address.sub('', tweet)
#     tweet = user.sub('', tweet)
#     return tweet

In [10]:
# output_dir = "/kaggle/working/outputs/formatted_data/er_clean"

# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# new_train_df = train_df.copy()
# new_train_df['tweet'] = train_df['tweet'].apply(lambda x: cleaner(x))
# new_train_df['tweet'] = train_df['tweet'].apply(lambda x: emoji.demojize(x, delimiters=("", "")))
# new_train_df.to_csv(f"{output_dir}/ha_train_er_clean.tsv", sep = '\t', index = False)

# new_dev_df = dev_df.copy()
# new_dev_df['tweet'] = dev_df['tweet'].apply(lambda x: cleaner(x))
# # new_dev_df['tweet'] = dev_df['tweet'].apply(lambda x: emoji.demojize(x, delimiters=("", "")))
# new_dev_df.to_csv(f"{output_dir}/ha_dev_er_clean.tsv", sep = '\t', index = False)

# new_test_df = test_df.copy()
# new_test_df['tweet'] = test_df['tweet'].apply(lambda x: cleaner(x))
# # new_test_df['tweet'] = test_df['tweet'].apply(lambda x: emoji.demojize(x, delimiters=("", "")))
# new_test_df.to_csv(f"{output_dir}/ha_test_er_clean.tsv", sep = '\t', index = False)

In [11]:
!ls full_outputs/*/* | wc -l

195


In [12]:
# from IPython.display import FileLink

# !tar -czvf modified_data.tar.gz outputs/formatted_data
# FileLink(r'modified_data.tar.gz')